In [1]:
import nltk
import math
nltk.download('stopwords')
import pandas as pd
import pickle
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
import networkx.algorithms.community as nx_community
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
import networkx.algorithms.community as nx_comm
import netwulf
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fridajorgensen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Nedenstående kode outputter noget, men den gør det ud fra tag og alle navne har desværre ikke navne tags :'(
# Så noget andet må gøres...

url = "https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the character names using the tag and class
characters = []
for element in soup.find_all('a', class_='mw-redirect'):
    name = element.text
    # Exclude characters that are not actual character names
    if name.startswith('List') or name.startswith('Category'):
        continue
    characters.append(name)

print(characters)
